In [1]:
# install all the data
!gcloud auth login
bucket_name = 'lily-li-dataset'
!mkdir -p /root/.cache/torch/mmf/data/datasets/satire
!gsutil -mq cp gs://{bucket_name}/satire/**.jsonl /content/

In [2]:
!pip install tokenizers transformers jsonlines

     |████████████████████████████████| 3.0MB 3.4MB/s 
     |████████████████████████████████| 778kB 17.4MB/s 
     |████████████████████████████████| 1.1MB 15.5MB/s 
     |████████████████████████████████| 890kB 15.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=053be86c10ad8d56ff73533451436d64372d29f089883b4ce34c4032f032c2d3
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
ERROR: transformers 3.0.2 has requirement tokenizers==0.8.1.rc1, but you'll have tokenizers 0.8.1 which is incompatible.


In [3]:
import jsonlines
import numpy as np

with jsonlines.open('train.jsonl', 'r') as data:
    data = list(data)
    X_train = np.array([d["text"] for d in data])
    y_train = np.array([d["label"] for d in data])

with jsonlines.open('test.jsonl', 'r') as data:
    data = list(data)
    X_test = np.array([d["text"] for d in data])
    y_test = np.array([d["label"] for d in data])

In [4]:
import os
import re
import json
import string
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig

configuration = BertConfig()  # default paramters and configuration for BERT

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [6]:
train_dict = tokenizer(list(X_train), padding="max_length", truncation=True, max_length=128, return_tensors="tf")
train_input_ids, train_segment_ids, train_attention_masks = (train_dict["input_ids"],
                                                             train_dict["token_type_ids"],
                                                             train_dict["attention_mask"])
test_dict = tokenizer(list(X_test), padding="max_length", truncation=True, max_length=128, return_tensors="tf")
test_input_ids, test_segment_ids, test_attention_masks = (test_dict["input_ids"],
                                                             test_dict["token_type_ids"],
                                                             test_dict["attention_mask"])

In [7]:
def create_model(input_shape=(128,)):
    encoder = TFBertModel.from_pretrained("bert-base-uncased")  

    input_ids = tf.keras.layers.Input(shape=(128,), dtype=tf.int32)
    segment_ids = tf.keras.layers.Input(shape=(128,), dtype=tf.int32)
    input_mask = tf.keras.layers.Input(shape=(128,), dtype=tf.int32)

    bert_output = encoder(input_ids, token_type_ids=segment_ids, attention_mask=input_mask)[0]
    bert_output = tf.keras.layers.Flatten()(bert_output)

    output = tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid)(bert_output)

    model = tf.keras.Model(
        inputs=[input_ids, segment_ids, input_mask],
        outputs=output,
    )
    return model

In [8]:
# Create distribution strategy
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)

# Create model
with strategy.scope():
    model = create_model()

INFO:tensorflow:Initializing the TPU system: grpc://10.59.28.98:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.59.28.98:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertModel for predictions without further training.


In [9]:
optimizer = tf.keras.optimizers.Adam(lr=5e-5)
model.compile(optimizer=optimizer, loss=tf.keras.losses.binary_crossentropy,
              metrics=["accuracy", "AUC", "Precision", "Recall"])

In [10]:
model.fit(x=[train_input_ids, train_segment_ids, train_attention_masks],
           y=y_train, batch_size=32, epochs=10, validation_steps=2000//32,
           steps_per_epoch=8000//32,
           validation_data=([test_input_ids, test_segment_ids, test_attention_masks], y_test))

Epoch 1/10


  2/250 [..............................] - ETA: 42:31 - loss: 0.6401 - accuracy: 0.6719 - auc: 0.6781 - precision: 0.6061 - recall: 0.7143  WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0069s vs `on_train_batch_end` time: 0.0896s). Check your callbacks.


250/250 [==============================] - ETA: 0s - loss: 0.3601 - accuracy: 0.8466 - auc: 0.9179 - precision: 0.8213 - recall: 0.7887WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0039s vs `on_test_batch_end` time: 0.0331s). Check your callbacks.


250/250 [==============================] - 58s 230ms/step - loss: 0.3601 - accuracy: 0.8466 - auc: 0.9179 - precision: 0.8213 - recall: 0.7887 - val_loss: 0.2320 - val_accuracy: 0.9133 - val_auc: 0.9677 - val_precision: 0.9294 - val_recall: 0.8472
Epoch 2/10
250/250 [==============================] - 27s 107ms/step - loss: 0.1272 - accuracy: 0.9559 - auc: 0.9880 - precision: 0.9478 - recall: 0.9416 - val_loss: 0.3447 - val_accuracy: 0.9123 - val_auc: 0.9643 - val_precision: 0.8576 - val_recall: 0.9356
Epoch 3/10
250/250 [==============================] - 27s 108ms/step - loss: 0.0648 - accuracy: 0.9788 - auc: 0.9965 - precision: 0.9741 - recall: 0.9728 - val_loss: 0.3274 - val_accuracy: 0.9068 - val_auc: 0.9610 - val_precision: 0.8550 - val_recall: 0.9230
Epoch 4/10
250/250 [==============================] - 27s 107ms/step - loss: 0.0508 - accuracy: 0.9823 - auc: 0.9977 - precision: 0.9796 - recall: 0.9760 - val_loss: 0.4260 - val_accuracy: 0.9002 - val_auc: 0.9546 - val_precision: 0.8